In [4]:
# Функция Розенброка в 2D
function rosenbrock_2d(x)
    return 100 * (x[2] - x[1]^2)^2 + (x[1] - 1)^2
end

# Функция Растригина в 2D
function rastrigin_2d(x)
    return 20 + x[1]^2 + x[2]^2 - 10 * (cos(2π * x[1]) + cos(2π * x[2]))
end

# Функция Экли в 2D
function ackley_2d(x)
    return -20 * exp(-0.2 * sqrt(0.5 * (x[1]^2 + x[2]^2))) -
           exp(0.5 * (cos(2π * x[1]) + cos(2π * x[2]))) + 20 + exp(1)
end

# Функция с несколькими локальными минимумами
function multimodal_2d(x)
    return sin(x[1]) * cos(x[2]) + 0.1 * (x[1]^2 + x[2]^2)
end

multimodal_2d (generic function with 1 method)

In [16]:
function compare_pso_variants_2d(objective, bounds;
                               num_particles=30, max_iter=10,
                               save_animation=false)

    dimensions = 2

    # Инициализация для классического PSO
    positions_classic = rand(dimensions, num_particles) .* (bounds[2] - bounds[1]) .+ bounds[1]
    velocities_classic = zeros(dimensions, num_particles)
    pbest_positions_classic = copy(positions_classic)
    pbest_values_classic = [objective(positions_classic[:, i]) for i in 1:num_particles]
    gbest_idx_classic = argmin(pbest_values_classic)
    gbest_position_classic = copy(pbest_positions_classic[:, gbest_idx_classic])
    gbest_value_classic = pbest_values_classic[gbest_idx_classic]
    prev_gbest_value_classic = -1000
    flag_classic = false
    iters_classic = 0

    # Инициализация для PSO с инерцией
    positions_inertia = copy(positions_classic)
    velocities_inertia = zeros(dimensions, num_particles)
    pbest_positions_inertia = copy(positions_inertia)
    pbest_values_inertia = [objective(positions_inertia[:, i]) for i in 1:num_particles]
    gbest_idx_inertia = argmin(pbest_values_inertia)
    gbest_position_inertia = copy(pbest_positions_inertia[:, gbest_idx_inertia])
    gbest_value_inertia = pbest_values_inertia[gbest_idx_inertia]
    prev_gbest_value_inertia = -1000
    flag_inertia = false

    # Инициализация для PSO с телепортацией
    positions_teleport = copy(positions_classic)
    velocities_teleport = zeros(dimensions, num_particles)
    pbest_positions_teleport = copy(positions_teleport)
    pbest_values_teleport = [objective(positions_teleport[:, i]) for i in 1:num_particles]
    gbest_idx_teleport = argmin(pbest_values_teleport)
    gbest_position_teleport = copy(pbest_positions_teleport[:, gbest_idx_teleport])
    gbest_value_teleport = pbest_values_teleport[gbest_idx_teleport]
    prev_gbest_value_teleport = -1000
    flag_teleport = false

    # Создание сетки для контурного графика
    grid_size = 100
    x = range(bounds[1], bounds[2], length=grid_size)
    y = range(bounds[1], bounds[2], length=grid_size)
    z = [objective([i, j]) for j in y, i in x]

    # Параметры для алгоритмов
    w_classic = 0.5
    c1_classic = 1.5
    c2_classic = 1.5

    w_start = 0.9
    w_end = 0.4
    c1_inertia = 1.5
    c2_inertia = 1.5

    w_teleport = 0.5
    c1_teleport = 1.5
    c2_teleport = 1.5
    teleport_prob = 0.05
    teleport_radius = 0.5

    # Создание анимации
    anim = @animate for iter in 1:max_iter
        # Создаем 2x2 сетку графиков
        p = plot(layout=(2, 2), size=(1200, 1000))

        # Контурный график функции (для справки)
        contour!(p[1], x, y, z, levels=20, color=:viridis,
                title="Целевая функция",
                xlabel="x", ylabel="y", colorbar=false)

        # Классический PSO
        contour!(p[2], x, y, z, levels=20, color=:viridis,
                title="PSO с инерцией: $(round(gbest_value_classic, digits=6))",
                xlabel="x", ylabel="y", colorbar=false)
        scatter!(p[2], positions_classic[1, :], positions_classic[2, :],
                label="", color=:white, markerstrokecolor=:black)
        scatter!(p[2], [gbest_position_classic[1]], [gbest_position_classic[2]],
                label="", color=:red, markersize=8)

        # PSO с инерцией
        contour!(p[3], x, y, z, levels=20, color=:viridis,
                title="Классический PSO: $(round(gbest_value_inertia, digits=6))",
                xlabel="x", ylabel="y", colorbar=false)
        scatter!(p[3], positions_inertia[1, :], positions_inertia[2, :],
                label="", color=:white, markerstrokecolor=:black)
        scatter!(p[3], [gbest_position_inertia[1]], [gbest_position_inertia[2]],
                label="", color=:red, markersize=8)

        # PSO с телепортацией
        contour!(p[4], x, y, z, levels=20, color=:viridis,
                title="PSO с телепортацией: $(round(gbest_value_teleport, digits=6))",
                xlabel="x", ylabel="y", colorbar=false)
        scatter!(p[4], positions_teleport[1, :], positions_teleport[2, :],
                label="", color=:white, markerstrokecolor=:black)
        scatter!(p[4], [gbest_position_teleport[1]], [gbest_position_teleport[2]],
                label="", color=:red, markersize=8)

        plot!(p, plot_title="Итерация $iter", plot_titlevspan=0.1)

        # Обновление для классического PSO
        if abs(gbest_value_classic - prev_gbest_value_classic) >= 1e-5
            for i in 1:num_particles
                r1, r2 = rand(dimensions), rand(dimensions)
                velocities_classic[:, i] = w_classic * velocities_classic[:, i] +
                                        c1_classic * r1 .* (pbest_positions_classic[:, i] - positions_classic[:, i]) +
                                        c2_classic * r2 .* (gbest_position_classic - positions_classic[:, i])
                positions_classic[:, i] += velocities_classic[:, i]
                positions_classic[:, i] = max.(bounds[1], min.(bounds[2], positions_classic[:, i]))

                current_value = objective(positions_classic[:, i])
                if current_value < pbest_values_classic[i]
                    pbest_values_classic[i] = current_value
                    pbest_positions_classic[:, i] = positions_classic[:, i]

                    if current_value < gbest_value_classic
                        prev_gbest_value_classic = gbest_value_classic
                        gbest_value_classic = current_value
                        gbest_position_classic = copy(positions_classic[:, i])
                    end
                end
            end
        else
            if !flag_classic
                println("Инерционный закончен", iter)  
            end  
            flag_classic=true
        end

        if abs(gbest_value_inertia - prev_gbest_value_inertia) >= 1e-5
            w_inertia = w_start - (w_start - w_end) * iter / max_iter
            for i in 1:num_particles
                r1, r2 = rand(dimensions), rand(dimensions)
                velocities_inertia[:, i] = w_inertia * velocities_inertia[:, i] +
                                        c1_inertia * r1 .* (pbest_positions_inertia[:, i] - positions_inertia[:, i]) +
                                        c2_inertia * r2 .* (gbest_position_inertia - positions_inertia[:, i])
                positions_inertia[:, i] += velocities_inertia[:, i]
                positions_inertia[:, i] = max.(bounds[1], min.(bounds[2], positions_inertia[:, i]))

                current_value = objective(positions_inertia[:, i])
                if current_value < pbest_values_inertia[i]
                    pbest_values_inertia[i] = current_value
                    pbest_positions_inertia[:, i] = positions_inertia[:, i]

                    if current_value < gbest_value_inertia
                        prev_gbest_value_inertia = gbest_value_inertia
                        gbest_value_inertia = current_value
                        gbest_position_inertia = copy(positions_inertia[:, i])
                    end
                end
            end
        else
            if !flag_inertia
                println("Классический закончен", iter)  
            end  
            flag_inertia=true
        end
        

        # Обновление для PSO с телепортацией
        if abs(gbest_value_teleport - prev_gbest_value_teleport) >= 1e-5
            for i in 1:num_particles
                if rand() < teleport_prob
                    positions_teleport[:, i] = gbest_position_teleport + teleport_radius * (2 * rand(dimensions) .- 1)
                    velocities_teleport[:, i] = zeros(dimensions)
                else
                    r1, r2 = rand(dimensions), rand(dimensions)
                    velocities_teleport[:, i] = w_teleport * velocities_teleport[:, i] +
                                            c1_teleport * r1 .* (pbest_positions_teleport[:, i] - positions_teleport[:, i]) +
                                            c2_teleport * r2 .* (gbest_position_teleport - positions_teleport[:, i])
                    positions_teleport[:, i] += velocities_teleport[:, i]
                end

                positions_teleport[:, i] = max.(bounds[1], min.(bounds[2], positions_teleport[:, i]))

                current_value = objective(positions_teleport[:, i])
                if current_value < pbest_values_teleport[i]
                    pbest_values_teleport[i] = current_value
                    pbest_positions_teleport[:, i] = positions_teleport[:, i]

                    if current_value < gbest_value_teleport
                        prev_gbest_value_teleport = gbest_value_teleport
                        gbest_value_teleport = current_value
                        gbest_position_teleport = copy(positions_teleport[:, i])
                    end
                end
            end
        else
            if !flag_teleport
                println("Телепорт закончен", iter)  
            end  
            flag_teleport=true
        end
        if flag_teleport && flag_inertia && flag_classic
            break
        end
    end

    if save_animation
        gif(anim, "pso_comparison.gif", fps=10)
    end

    return anim
end

# Сравнение различных вариантов PSO на функции Растригина
anim_comparison = compare_pso_variants_2d(rastrigin_2d, (-5.12, 5.12),
                                        num_particles=30, max_iter=100,
                                        save_animation=true)

Телепорт закончен31
Инерционный закончен33
Классический закончен46


┌ Info: Saved animation to /Users/d.okutin/pso_comparison.gif
└ @ Plots /Users/d.okutin/.julia/packages/Plots/kLeqV/src/animation.jl:156


Animation("/var/folders/r_/mggr546j3qjgz1bx6blb249h0000gq/T/jl_cOvdQU", ["000001.png", "000002.png", "000003.png", "000004.png", "000005.png", "000006.png", "000007.png", "000008.png", "000009.png", "000010.png"  …  "000036.png", "000037.png", "000038.png", "000039.png", "000040.png", "000041.png", "000042.png", "000043.png", "000044.png", "000045.png"])

In [11]:
anim_comparison = compare_pso_variants_2d(rosenbrock_2d, (-5.12, 5.12),
                                        num_particles=30, max_iter=100,
                                        save_animation=true)

Классический закончен35
Инерционный закончен86


┌ Info: Saved animation to /Users/d.okutin/pso_comparison.gif
└ @ Plots /Users/d.okutin/.julia/packages/Plots/kLeqV/src/animation.jl:156


Animation("/var/folders/r_/mggr546j3qjgz1bx6blb249h0000gq/T/jl_64MaxC", ["000001.png", "000002.png", "000003.png", "000004.png", "000005.png", "000006.png", "000007.png", "000008.png", "000009.png", "000010.png"  …  "000091.png", "000092.png", "000093.png", "000094.png", "000095.png", "000096.png", "000097.png", "000098.png", "000099.png", "000100.png"])

In [17]:
function run_classic_pso(objective, dimensions, bounds, num_particles, max_iter, tolerance=1e-6, max_stagnation=20)
    # Параметры алгоритма
    w_classic = 0.5      # вес инерции
    c1_classic = 1.5     # коэффициент когнитивной составляющей
    c2_classic = 1.5     # коэффициент социальной составляющей
    
    # Инициализация позиций и скоростей частиц
    positions_classic = rand(dimensions, num_particles) .* (bounds[2] - bounds[1]) .+ bounds[1]
    velocities_classic = zeros(dimensions, num_particles)
    
    # Лучшие позиции частиц и их значения
    pbest_positions_classic = copy(positions_classic)
    pbest_values_classic = [objective(positions_classic[:, i]) for i in 1:num_particles]
    
    # Глобальное лучшее значение и позиция
    gbest_idx = argmin(pbest_values_classic)
    gbest_position_classic = copy(positions_classic[:, gbest_idx])
    gbest_value_classic = pbest_values_classic[gbest_idx]
    
    # Для отслеживания стагнации
    stagnation_counter = 0
    prev_gbest_value = gbest_value_classic
    
    # История для анимации
    history = [(copy(positions_classic), copy(gbest_position_classic), gbest_value_classic)]
    
    # Основной цикл
    for iter in 1:max_iter
        # Обновление для классического PSO
        for i in 1:num_particles
            r1, r2 = rand(dimensions), rand(dimensions)
            velocities_classic[:, i] = w_classic * velocities_classic[:, i] +
                                      c1_classic * r1 .* (pbest_positions_classic[:, i] - positions_classic[:, i]) +
                                      c2_classic * r2 .* (gbest_position_classic - positions_classic[:, i])
            positions_classic[:, i] += velocities_classic[:, i]
            positions_classic[:, i] = max.(bounds[1], min.(bounds[2], positions_classic[:, i]))

            current_value = objective(positions_classic[:, i])
            if current_value < pbest_values_classic[i]
                pbest_values_classic[i] = current_value
                pbest_positions_classic[:, i] = positions_classic[:, i]

                if current_value < gbest_value_classic
                    gbest_value_classic = current_value
                    gbest_position_classic = copy(positions_classic[:, i])
                end
            end
        end
        
        # Добавление текущего состояния в историю
        push!(history, (copy(positions_classic), copy(gbest_position_classic), gbest_value_classic))
        
        # Проверка на стагнацию
        if abs(gbest_value_classic - prev_gbest_value) < tolerance
            stagnation_counter += 1
            if stagnation_counter >= max_stagnation
                println("Останов по стагнации на итерации $iter")
                break
            end
        else
            stagnation_counter = 0
            prev_gbest_value = gbest_value_classic
        end
    end
    
    return history, gbest_position_classic, gbest_value_classic
end

function run_inertia_pso(objective, dimensions, bounds, num_particles, max_iter, tolerance=1e-6, max_stagnation=20)
    # Параметры алгоритма
    w_start = 0.9        # начальный вес инерции
    w_end = 0.4          # конечный вес инерции
    c1_inertia = 1.5     # коэффициент когнитивной составляющей
    c2_inertia = 1.5     # коэффициент социальной составляющей
    
    # Инициализация позиций и скоростей частиц
    positions_inertia = rand(dimensions, num_particles) .* (bounds[2] - bounds[1]) .+ bounds[1]
    velocities_inertia = zeros(dimensions, num_particles)
    
    # Лучшие позиции частиц и их значения
    pbest_positions_inertia = copy(positions_inertia)
    pbest_values_inertia = [objective(positions_inertia[:, i]) for i in 1:num_particles]
    
    # Глобальное лучшее значение и позиция
    gbest_idx = argmin(pbest_values_inertia)
    gbest_position_inertia = copy(positions_inertia[:, gbest_idx])
    gbest_value_inertia = pbest_values_inertia[gbest_idx]
    
    # Для отслеживания стагнации
    stagnation_counter = 0
    prev_gbest_value = gbest_value_inertia
    
    # История для анимации
    history = [(copy(positions_inertia), copy(gbest_position_inertia), gbest_value_inertia)]
    
    # Основной цикл
    for iter in 1:max_iter
        # Обновление веса инерции
        w_inertia = w_start - (w_start - w_end) * iter / max_iter
        
        # Обновление для PSO с инерцией
        for i in 1:num_particles
            r1, r2 = rand(dimensions), rand(dimensions)
            velocities_inertia[:, i] = w_inertia * velocities_inertia[:, i] +
                                      c1_inertia * r1 .* (pbest_positions_inertia[:, i] - positions_inertia[:, i]) +
                                      c2_inertia * r2 .* (gbest_position_inertia - positions_inertia[:, i])
            positions_inertia[:, i] += velocities_inertia[:, i]
            positions_inertia[:, i] = max.(bounds[1], min.(bounds[2], positions_inertia[:, i]))

            current_value = objective(positions_inertia[:, i])
            if current_value < pbest_values_inertia[i]
                pbest_values_inertia[i] = current_value
                pbest_positions_inertia[:, i] = positions_inertia[:, i]

                if current_value < gbest_value_inertia
                    gbest_value_inertia = current_value
                    gbest_position_inertia = copy(positions_inertia[:, i])
                end
            end
        end
        
        # Добавление текущего состояния в историю
        push!(history, (copy(positions_inertia), copy(gbest_position_inertia), gbest_value_inertia))
        
        # Проверка на стагнацию
        if abs(gbest_value_inertia - prev_gbest_value) < tolerance
            stagnation_counter += 1
            if stagnation_counter >= max_stagnation
                println("Останов по стагнации на итерации $iter")
                break
            end
        else
            stagnation_counter = 0
            prev_gbest_value = gbest_value_inertia
        end
    end
    
    return history, gbest_position_inertia, gbest_value_inertia
end

function run_teleport_pso(objective, dimensions, bounds, num_particles, max_iter, teleport_prob=0.05, teleport_radius=0.5, tolerance=1e-6, max_stagnation=20)
    # Параметры алгоритма
    w_teleport = 0.5     # вес инерции
    c1_teleport = 1.5    # коэффициент когнитивной составляющей
    c2_teleport = 1.5    # коэффициент социальной составляющей
    
    # Инициализация позиций и скоростей частиц
    positions_teleport = rand(dimensions, num_particles) .* (bounds[2] - bounds[1]) .+ bounds[1]
    velocities_teleport = zeros(dimensions, num_particles)
    
    # Лучшие позиции частиц и их значения
    pbest_positions_teleport = copy(positions_teleport)
    pbest_values_teleport = [objective(positions_teleport[:, i]) for i in 1:num_particles]
    
    # Глобальное лучшее значение и позиция
    gbest_idx = argmin(pbest_values_teleport)
    gbest_position_teleport = copy(positions_teleport[:, gbest_idx])
    gbest_value_teleport = pbest_values_teleport[gbest_idx]
    
    # Для отслеживания стагнации
    stagnation_counter = 0
    prev_gbest_value = gbest_value_teleport
    
    # История для анимации
    history = [(copy(positions_teleport), copy(gbest_position_teleport), gbest_value_teleport)]
    
    # Основной цикл
    for iter in 1:max_iter
        # Обновление для PSO с телепортацией
        for i in 1:num_particles
            if rand() < teleport_prob
                positions_teleport[:, i] = gbest_position_teleport + teleport_radius * (2 * rand(dimensions) .- 1)
                velocities_teleport[:, i] = zeros(dimensions)
            else
                r1, r2 = rand(dimensions), rand(dimensions)
                velocities_teleport[:, i] = w_teleport * velocities_teleport[:, i] +
                                          c1_teleport * r1 .* (pbest_positions_teleport[:, i] - positions_teleport[:, i]) +
                                          c2_teleport * r2 .* (gbest_position_teleport - positions_teleport[:, i])
                positions_teleport[:, i] += velocities_teleport[:, i]
            end

            positions_teleport[:, i] = max.(bounds[1], min.(bounds[2], positions_teleport[:, i]))

            current_value = objective(positions_teleport[:, i])
            if current_value < pbest_values_teleport[i]
                pbest_values_teleport[i] = current_value
                pbest_positions_teleport[:, i] = positions_teleport[:, i]

                if current_value < gbest_value_teleport
                    gbest_value_teleport = current_value
                    gbest_position_teleport = copy(positions_teleport[:, i])
                end
            end
        end
        
        # Добавление текущего состояния в историю
        push!(history, (copy(positions_teleport), copy(gbest_position_teleport), gbest_value_teleport))
        
        # Проверка на стагнацию
        if abs(gbest_value_teleport - prev_gbest_value) < tolerance
            stagnation_counter += 1
            if stagnation_counter >= max_stagnation
                println("Останов по стагнации на итерации $iter")
                break
            end
        else
            stagnation_counter = 0
            prev_gbest_value = gbest_value_teleport
        end
    end
    
    return history, gbest_position_teleport, gbest_value_teleport
end

# Основная функция для запуска всех алгоритмов и создания анимации
function run_pso_comparison(objective, dimensions, bounds, num_particles, max_iter)
    # Запуск всех трех вариантов PSO
    history_classic, gbest_position_classic, gbest_value_classic = run_classic_pso(objective, dimensions, bounds, num_particles, max_iter)
    history_inertia, gbest_position_inertia, gbest_value_inertia = run_inertia_pso(objective, dimensions, bounds, num_particles, max_iter)
    history_teleport, gbest_position_teleport, gbest_value_teleport = run_teleport_pso(objective, dimensions, bounds, num_particles, max_iter)
    
    # Определение максимального числа кадров для анимации
    max_frames = max(length(history_classic), length(history_inertia), length(history_teleport))
    
    # Подготовка сетки для контурного графика
    x = range(bounds[1], bounds[2], length=100)
    y = range(bounds[1], bounds[2], length=100)
    z = [objective([i, j]) for i in x, j in y]
    
    # Создание анимации
    anim = @animate for frame in 1:max_frames
        # Получение текущих данных для каждого алгоритма
        classic_frame = min(frame, length(history_classic))
        inertia_frame = min(frame, length(history_inertia))
        teleport_frame = min(frame, length(history_teleport))
        
        positions_classic, gbest_pos_classic, gbest_val_classic = history_classic[classic_frame]
        positions_inertia, gbest_pos_inertia, gbest_val_inertia = history_inertia[inertia_frame]
        positions_teleport, gbest_pos_teleport, gbest_val_teleport = history_teleport[teleport_frame]
        
        # Создаем 2x2 сетку графиков
        p = plot(layout=(2, 2), size=(1200, 1000))

        # Контурный график функции (для справки)
        contour!(p[1], x, y, z, levels=20, color=:viridis,
                title="Целевая функция",
                xlabel="x", ylabel="y", colorbar=false)

        # Классический PSO
        contour!(p[2], x, y, z, levels=20, color=:viridis,
                title="PSO с инерцией: $(round(gbest_val_classic, digits=6))",
                xlabel="x", ylabel="y", colorbar=false)
        scatter!(p[2], positions_classic[1, :], positions_classic[2, :],
                label="", color=:white, markerstrokecolor=:black)
        scatter!(p[2], [gbest_pos_classic[1]], [gbest_pos_classic[2]],
                label="", color=:red, markersize=8)

        # PSO с инерцией
        contour!(p[3], x, y, z, levels=20, color=:viridis,
                title="Классический PSO: $(round(gbest_val_inertia, digits=6))",
                xlabel="x", ylabel="y", colorbar=false)
        scatter!(p[3], positions_inertia[1, :], positions_inertia[2, :],
                label="", color=:white, markerstrokecolor=:black)
        scatter!(p[3], [gbest_pos_inertia[1]], [gbest_pos_inertia[2]],
                label="", color=:red, markersize=8)

        # PSO с телепортацией
        contour!(p[4], x, y, z, levels=20, color=:viridis,
                title="PSO с телепортацией: $(round(gbest_val_teleport, digits=6))",
                xlabel="x", ylabel="y", colorbar=false)
        scatter!(p[4], positions_teleport[1, :], positions_teleport[2, :],
                label="", color=:white, markerstrokecolor=:black)
        scatter!(p[4], [gbest_pos_teleport[1]], [gbest_pos_teleport[2]],
                label="", color=:red, markersize=8)

        plot!(p, plot_title="Итерация $frame", plot_titlevspan=0.1)
    end
    
    return anim, gbest_position_classic, gbest_value_classic, gbest_position_inertia, gbest_value_inertia, gbest_position_teleport, gbest_value_teleport
end

# Пример использования
function objective(x)
    return (1.0 - x[1])^2 + 100.0 * (x[2] - x[1]^2)^2  # Функция Розенброка
end

dimensions = 2
bounds = [-3.0, 3.0]
num_particles = 30
max_iter = 100

anim, gbest_pos_classic, gbest_val_classic, gbest_pos_inertia, gbest_val_inertia, gbest_pos_teleport, gbest_val_teleport = 
    run_pso_comparison(objective, dimensions, bounds, num_particles, max_iter)

# Сохранение анимации
gif(anim, "pso_comparison.gif", fps = 10)

# Вывод результатов
println("Классический PSO: позиция = $(gbest_pos_classic), значение = $(gbest_val_classic)")
println("PSO с инерцией: позиция = $(gbest_pos_inertia), значение = $(gbest_val_inertia)")
println("PSO с телепортацией: позиция = $(gbest_pos_teleport), значение = $(gbest_val_teleport)")

Останов по стагнации на итерации 47
Останов по стагнации на итерации 79
Классический PSO: позиция = [1.0005934044537512, 1.001220943654881], значение = 4.6625537722517884e-7
PSO с инерцией: позиция = [0.9953017904949121, 0.9907194615048974], значение = 2.2953154304757948e-5
PSO с телепортацией: позиция = [0.9999966471502021, 0.9999893203269791], значение = 1.590497014861752e-9


┌ Info: Saved animation to /Users/d.okutin/pso_comparison.gif
└ @ Plots /Users/d.okutin/.julia/packages/Plots/kLeqV/src/animation.jl:156
